In [ ]:
pip install pycoingecko

## PRIVATE UNLOCKS STRATEGY

In [ ]:
import pandas as pd
from pycoingecko import CoinGeckoAPI
import requests
from datetime import datetime, timedelta
from google.colab import auth
import gspread
from google.auth import default
from datetime import date

cg = CoinGeckoAPI()

def get_price_data(token_ids, days=5):
    price_data = {}
    for token_id in token_ids:
        try:
            data = cg.get_coin_market_chart_by_id(id=token_id, vs_currency='usd', days=days)
            df = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.set_index('timestamp', inplace=True)
            price_data[token_id] = df['price']
        except Exception as e:
            print(f"Error fetching data for {token_id}: {e}")
    return pd.DataFrame(price_data).resample('D').last().ffill()

start_time = datetime.now() - timedelta(days=5)
end_time = datetime.now()

coingecko_ids = ['ionic-protocol']
price_data = get_price_data(coingecko_ids)
latest_price = price_data.iloc[-1]['ionic-protocol']
auth.authenticate_user()

creds, _ = default()
access = gspread.authorize(creds)

# Open the workbook and sheet
wb_id_dest = '1tWPMKIqRxg_noABRmQLhti0qXwG3c8bM30bvdzvxruE'
sheet_id_dest = 987349207

wb_dest = access.open_by_key(wb_id_dest)
sheet_dest = wb_dest.get_worksheet_by_id(sheet_id_dest)

# Update cell B2 with the latest price
sheet_dest.update('C1', [[latest_price]])

today = date.today().strftime("%Y-%m-%d")
print(f"Latest price ({latest_price}) exported to cell C1. Last refresh {today}")

## DAILY PRICES SHEET

In [ ]:
# DAILY PRICES SHEET
import pandas as pd
from pycoingecko import CoinGeckoAPI
import requests
from datetime import datetime, timedelta
from google.colab import auth
import gspread
from google.auth import default
from datetime import date

cg = CoinGeckoAPI()

def get_price_data(token_ids, days=5):
    price_data = {}
    for token_id in token_ids:
        try:
            data = cg.get_coin_market_chart_by_id(id=token_id, vs_currency='usd', days=days)
            df = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.set_index('timestamp', inplace=True)
            price_data[token_id] = df['price']
        except Exception as e:
            print(f"Error fetching data for {token_id}: {e}")
    return pd.DataFrame(price_data).resample('D').last().ffill()

start_time = datetime.now() - timedelta(days=5)
end_time = datetime.now()

coingecko_ids = ['ionic-protocol','ethereum', 'aerodrome-finance']
price_data = get_price_data(coingecko_ids)


latest_price_ion = price_data.iloc[-1]['ionic-protocol']
latest_price_eth = price_data.iloc[-1]['ethereum']
latest_price_aero = price_data.iloc[-1]['aerodrome-finance']

auth.authenticate_user()
creds, _ = default()
access = gspread.authorize(creds)

# Open the workbook and sheet
wb_id_dest = '1tWPMKIqRxg_noABRmQLhti0qXwG3c8bM30bvdzvxruE'
sheet_id_dest = 2011546042

wb_dest = access.open_by_key(wb_id_dest)
sheet_dest = wb_dest.get_worksheet_by_id(sheet_id_dest)

# Update cell B1+B4 with the latest prices
sheet_dest.update('B2', [[latest_price_ion]])
sheet_dest.update('B3', [[latest_price_eth]])
sheet_dest.update('B4', [[latest_price_aero]])

today = date.today().strftime("%Y-%m-%d")
print(f"Latest prices exported to cell B2, B3, and B4. Last refresh {today}")

# DAILY PRICES 30D MOVING AVGS

In [ ]:
# DAILY PRICES SHEET
# Get 30 days of price data
coingecko_ids = ['ionic-protocol', 'ethereum', 'aerodrome-finance']
price_data = get_price_data(coingecko_ids, days=30)

# Calculate 30-day moving averages and percent changes for each token
results = {}
for token in coingecko_ids:
    # Calculate moving average
    moving_average = price_data[token].mean()

    # Calculate percent change
    first_price = price_data[token].iloc[0]
    last_price = price_data[token].iloc[-1]
    percent_change = ((last_price - first_price) / first_price) * 100

    results[token] = {
        'ma': moving_average,
        'pct': f"{percent_change:.2f}%"
    }

# Open the workbook and sheet
wb_id_dest = '1tWPMKIqRxg_noABRmQLhti0qXwG3c8bM30bvdzvxruE'
sheet_id_dest = 2011546042

wb_dest = access.open_by_key(wb_id_dest)
sheet_dest = wb_dest.get_worksheet_by_id(sheet_id_dest)

# Update cells with values (ION in C2/D2, ETH in C3/D3, AERO in C4/D4)
sheet_dest.update('C2:D4', [
    [results['ionic-protocol']['ma'], results['ionic-protocol']['pct']],
    [results['ethereum']['ma'], results['ethereum']['pct']],
    [results['aerodrome-finance']['ma'], results['aerodrome-finance']['pct']]
])

today = date.today().strftime("%Y-%m-%d")
print(f"30-day moving averages and percent changes exported for ION, ETH, and AERO. Last refresh {today}")

## Other sheets

In [ ]:
# Get 30 days of price data
coingecko_ids = ['ionic-protocol']
price_data = get_price_data(coingecko_ids, days=30)

# Calculate 30-day moving average
moving_average = price_data['ionic-protocol'].mean()

# Calculate percent change
first_price = price_data['ionic-protocol'].iloc[0]  # First day's price
last_price = price_data['ionic-protocol'].iloc[-1]  # Last day's price
percent_change = ((last_price - first_price) / first_price) * 100

# Open the workbook and sheet
wb_id_dest = '1tWPMKIqRxg_noABRmQLhti0qXwG3c8bM30bvdzvxruE'
sheet_id_dest = 987349207

wb_dest = access.open_by_key(wb_id_dest)
sheet_dest = wb_dest.get_worksheet_by_id(sheet_id_dest)

# Update cells with values
sheet_dest.update('I1', [[moving_average]])

# Format percent change and apply conditional formatting
formatted_percent = f"{percent_change:.2f}%"
sheet_dest.update('K1', [[formatted_percent]])


today = date.today().strftime("%Y-%m-%d")
print(f"30-day moving average ({moving_average}) and percent change ({formatted_percent}) exported. Last refresh {today}")

## AERO PRICE

In [ ]:
# Get current price data for AERO
coingecko_ids = ['aerodrome-finance']
price_data = get_price_data(coingecko_ids, days=1)  # Just need current price

# Get the latest price
latest_price = price_data['aerodrome-finance'].iloc[-1]

# Open the workbook and sheet
wb_id_dest = '1tWPMKIqRxg_noABRmQLhti0qXwG3c8bM30bvdzvxruE'
sheet_id_dest = 987349207

wb_dest = access.open_by_key(wb_id_dest)
sheet_dest = wb_dest.get_worksheet_by_id(sheet_id_dest)

# Update cell with the latest price
sheet_dest.update('E1', [[latest_price]])

today = date.today().strftime("%Y-%m-%d")
print(f"Latest AERO price ({latest_price}) exported to cell E1. Last refresh {today}")

## AERO POL STRATEGY sheet updates

In [ ]:
import pandas as pd
from pycoingecko import CoinGeckoAPI
import requests
from datetime import datetime, timedelta
from google.colab import auth
import gspread
from google.auth import default
from datetime import date

cg = CoinGeckoAPI()

def get_price_data(token_ids, days=5):
    price_data = {}
    for token_id in token_ids:
        try:
            data = cg.get_coin_market_chart_by_id(id=token_id, vs_currency='usd', days=days)
            df = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.set_index('timestamp', inplace=True)
            price_data[token_id] = df['price']
        except Exception as e:
            print(f"Error fetching data for {token_id}: {e}")
    return pd.DataFrame(price_data).resample('D').last().ffill()

start_time = datetime.now() - timedelta(days=5)
end_time = datetime.now()

coingecko_ids = ['ionic-protocol','ethereum', 'aerodrome-finance']
price_data = get_price_data(coingecko_ids)


latest_price_ion = price_data.iloc[-1]['ionic-protocol']
latest_price_eth = price_data.iloc[-1]['ethereum']
latest_price_aero = price_data.iloc[-1]['aerodrome-finance']

auth.authenticate_user()

creds, _ = default()
access = gspread.authorize(creds)

# Open the workbook and sheet
wb_id_dest = '1tWPMKIqRxg_noABRmQLhti0qXwG3c8bM30bvdzvxruE'
sheet_id_dest = 1677780990

wb_dest = access.open_by_key(wb_id_dest)
sheet_dest = wb_dest.get_worksheet_by_id(sheet_id_dest)

# Update cell B1+B4 with the latest prices
sheet_dest.update('B1', [[latest_price_ion]])
sheet_dest.update('B4', [[latest_price_eth]])
sheet_dest.update('D12', [[latest_price_aero]])

today = date.today().strftime("%Y-%m-%d")
print(f"Latest price ({latest_price}) exported to cell B1, B4, and D12. Last refresh {today}")